# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 16 2022 11:03AM,246616,12,U1027,"Uniderm USA, Inc",Released
1,Sep 16 2022 10:58AM,246615,12,HH8030,Hush Hush,Released
2,Sep 16 2022 10:58AM,246615,12,HH8033,Hush Hush,Released
3,Sep 16 2022 10:46AM,246613,20,ALU0007939,Alumier Labs Inc.,Released
4,Sep 16 2022 10:46AM,246614,21,409206602,"NBTY Global, Inc.",Released
5,Sep 16 2022 10:45AM,246612,20,8531127,"ACI Healthcare USA, Inc.",Released
6,Sep 16 2022 10:45AM,246612,20,8533773,"ACI Healthcare USA, Inc.",Executing
7,Sep 16 2022 10:32AM,246609,19,ADV80005705,"AdvaGen Pharma, Ltd",Executing
8,Sep 16 2022 10:32AM,246609,19,ADV80005706,"AdvaGen Pharma, Ltd",Executing
9,Sep 16 2022 10:32AM,246609,19,ADV80005707,"AdvaGen Pharma, Ltd",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,246612,Released,1
28,246613,Released,1
29,246614,Released,1
30,246615,Released,2
31,246616,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246612,NaN,1.0,1.0
246613,NaN,NaN,1.0
246614,NaN,NaN,1.0
246615,NaN,NaN,2.0
246616,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246294,0.0,1.0,0.0
246296,0.0,1.0,0.0
246517,0.0,0.0,1.0
246528,0.0,0.0,2.0
246532,1.0,9.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246294,0,1,0
246296,0,1,0
246517,0,0,1
246528,0,0,2
246532,1,9,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246294,0,1,0
1,246296,0,1,0
2,246517,0,0,1
3,246528,0,0,2
4,246532,1,9,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246294,,1,
1,246296,,1,
2,246517,,,1
3,246528,,,2
4,246532,1,9,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc"
1,Sep 16 2022 10:58AM,246615,12,Hush Hush
3,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.
4,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc."
5,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc."
7,Sep 16 2022 10:32AM,246609,19,"AdvaGen Pharma, Ltd"
15,Sep 16 2022 10:27AM,246607,10,Bio-PRF
26,Sep 16 2022 10:20AM,246605,21,"NBTY Global, Inc."
27,Sep 16 2022 10:13AM,246604,21,"NBTY Global, Inc."
28,Sep 16 2022 10:10AM,246602,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc",,,1
1,Sep 16 2022 10:58AM,246615,12,Hush Hush,,,2
2,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.,,,1
3,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc.",,,1
4,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc.",,1,1
5,Sep 16 2022 10:32AM,246609,19,"AdvaGen Pharma, Ltd",,7,1
6,Sep 16 2022 10:27AM,246607,10,Bio-PRF,,,11
7,Sep 16 2022 10:20AM,246605,21,"NBTY Global, Inc.",,,1
8,Sep 16 2022 10:13AM,246604,21,"NBTY Global, Inc.",,,1
9,Sep 16 2022 10:10AM,246602,12,"Uniderm USA, Inc",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc",1,,
1,Sep 16 2022 10:58AM,246615,12,Hush Hush,2,,
2,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.,1,,
3,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc.",1,,
4,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc.",1,1,
5,Sep 16 2022 10:32AM,246609,19,"AdvaGen Pharma, Ltd",1,7,
6,Sep 16 2022 10:27AM,246607,10,Bio-PRF,11,,
7,Sep 16 2022 10:20AM,246605,21,"NBTY Global, Inc.",1,,
8,Sep 16 2022 10:13AM,246604,21,"NBTY Global, Inc.",1,,
9,Sep 16 2022 10:10AM,246602,12,"Uniderm USA, Inc",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc",1,,
1,Sep 16 2022 10:58AM,246615,12,Hush Hush,2,,
2,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.,1,,
3,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc.",1,,
4,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc.",1,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc",1.0,NaN,NaN
1,Sep 16 2022 10:58AM,246615,12,Hush Hush,2.0,NaN,NaN
2,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.,1.0,NaN,NaN
3,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc.",1.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 16 2022 11:03AM,246616,12,"Uniderm USA, Inc",1.0,0.0,0.0
1,Sep 16 2022 10:58AM,246615,12,Hush Hush,2.0,0.0,0.0
2,Sep 16 2022 10:46AM,246613,20,Alumier Labs Inc.,1.0,0.0,0.0
3,Sep 16 2022 10:46AM,246614,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Sep 16 2022 10:45AM,246612,20,"ACI Healthcare USA, Inc.",1.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2219172,40.0,16.0,1.0
12,739833,6.0,0.0,0.0
16,986287,5.0,0.0,0.0
19,739787,1.0,9.0,0.0
20,493225,2.0,1.0,0.0
21,1232413,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2219172,40.0,16.0,1.0
1,12,739833,6.0,0.0,0.0
2,16,986287,5.0,0.0,0.0
3,19,739787,1.0,9.0,0.0
4,20,493225,2.0,1.0,0.0
5,21,1232413,3.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,16.0,1.0
1,12,6.0,0.0,0.0
2,16,5.0,0.0,0.0
3,19,1.0,9.0,0.0
4,20,2.0,1.0,0.0
5,21,3.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,6.0
2,16,Released,5.0
3,19,Released,1.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,0.0,9.0,1.0,2.0
Released,40.0,6.0,5.0,1.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,9.0,1.0,2.0
2,Released,40.0,6.0,5.0,1.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,9.0,1.0,2.0
2,Released,40.0,6.0,5.0,1.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()